In [ ]:
import pandas as pd
label = pd.read_csv("/home/iatell/projects/meta-learning/data/seq_line_labels.csv")
label["seq_len"] = label["endIndex"] - label["startIndex"]
label

In [2]:

import pandas as pd
df = pd.read_csv("/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles_prop.csv")
df

,timestamp,open,high,low,close,volume,upper_shadow,body,lower_shadow,Candle_Color,upper_body_ratio,lower_body_ratio,upper_lower_body_ratio
0,2018-01-01,13707.91,13818.55,12750.00,13380.00,8607.15640,0.076003,-0.225254,0.432772,1,0.337410,1.921259,0.175619
1,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.16540,0.540071,0.874627,0.332912,2,0.617487,0.380633,1.622262
2,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.48210,0.263644,0.155931,0.366880,2,1.690776,2.352839,0.718611
3,2018-01-04,14919.51,15280.00,13918.04,15059.54,25224.41500,0.150006,0.095280,0.681423,2,1.574377,5.000000,0.220136
4,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.35200,0.144690,1.274181,0.308056,2,0.113556,0.241768,0.469688
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,2022-05-19,28715.33,30545.18,28691.38,30319.23,67877.36415,0.109006,0.773779,0.011554,2,0.140875,0.014932,5.000000
1600,2022-05-20,30319.22,30777.33,28730.00,29201.01,60517.25325,0.221063,-0.539597,0.227288,1,0.409682,0.421218,0.972612
1601,2022-05-21,29201.01,29656.18,28947.28,29445.06,20987.13124,0.103235,0.119338,0.124071,2,0.865069,1.039664,0.832066
1602,2022-05-22,29445.07,30487.99,29255.11,30293.94,36158.98748,0.095648,0.418411,0.093632,2,0.228598,0.223780,1.021531


# model


In [1]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F


def mdn_split_params(raw_params, n_components):
    """
    raw_params: (B, 3K) tensor from mdn_head
    returns:
        pi    (B, K) mixture weights
        mu    (B, K) means
        sigma (B, K) std devs
    """
    B, threeK = raw_params.shape
    assert threeK == 3 * n_components

    raw = raw_params.view(B, n_components, 3)

    pi = raw[..., 0]                 # (B,K)
    mu = raw[..., 1]                 # (B,K)
    sigma = raw[..., 2]              # (B,K)

    pi = F.softmax(pi, dim=-1)       # weights sum to 1
    sigma = F.softplus(sigma) + 1e-4 # strictly positive
    return pi, mu, sigma


def mdn_nll_multitarget(y_line, pi, mu, sigma):
    """
    Negative log-likelihood for MDN with multiple valid targets per sample.
    Args:
        y_line : (B, L) padded targets (0 where invalid)
        pi, mu, sigma : (B, K) MDN params
    Returns:
        scalar loss
    """
    B, K = mu.shape
    losses = []

    for b in range(B):
        valid_y = y_line[b][y_line[b] > 0]  # (M,)
        if len(valid_y) == 0:
            continue

        # expand to (M, K)
        y_exp = valid_y.unsqueeze(-1).expand(-1, K)

        log_prob = -0.5 * ((y_exp - mu[b]) / (sigma[b] + 1e-8))**2 \
                   - torch.log(sigma[b] + 1e-8) \
                   - 0.5 * torch.log(torch.tensor(2.0 * torch.pi, device=y_line.device))

        log_mix = torch.log(pi[b] + 1e-8) + log_prob
        log_sum = torch.logsumexp(log_mix, dim=-1)  # (M,)

        losses.append(-log_sum.mean())

    if len(losses) == 0:
        return torch.tensor(0.0, device=y_line.device, requires_grad=True)

    return torch.stack(losses).mean()


class CNNLSTM_MDN(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, num_layers=1,
                 lr=1e-3, n_components=5, cnn_channels=64, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()

        # CNN
        self.conv1 = nn.Conv1d(input_dim, cnn_channels, kernel_size=1)
        self.conv3 = nn.Conv1d(input_dim, cnn_channels, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(cnn_channels)
        self.bn3   = nn.BatchNorm1d(cnn_channels)

        # LSTM
        fused_dim = 2 * cnn_channels
        self.lstm = nn.LSTM(fused_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # MDN output (single step)
        self.mdn_head = nn.Linear(hidden_dim, 3 * n_components)
        self.n_components = n_components
        self.lr = lr

    def forward(self, X, lengths=None):
        # CNN
        x = X["main"].transpose(1, 2)
        # x = X.transpose(1, 2)
        x1 = F.relu(self.bn1(self.conv1(x)))
        x3 = F.relu(self.bn3(self.conv3(x)))
        xf = torch.cat([x1, x3], dim=1).transpose(1, 2)

        # LSTM
        _, (h_last, _) = self.lstm(xf)
        last_h = h_last[-1]  # (B,H)

        # MDN (one step)
        raw = self.mdn_head(last_h)  # (B,3K)
        pi, mu, sigma = mdn_split_params(raw, self.n_components)
        return {"pi": pi, "mu": mu, "sigma": sigma}

    def training_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
        self.log("val/loss", loss, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)


# train

In [2]:
import sys
from pathlib import Path

# Current notebook location
notebook_path = Path().resolve()

# Add parent folder (meta/) to sys.path
sys.path.append(str(notebook_path.parent))
import joblib
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix,mean_absolute_error,mean_squared_error
from datetime import datetime
from preprocess.multi_regression_seq_dif import preprocess_sequences_csv_multilines
# from models.LSTM.lstm_multi_line_reg_seq_dif import LSTMMultiRegressor
from utils.print_batch import print_batch
from utils.to_address import to_address
from utils.json_to_csv import json_to_csv_in_memory
from utils.padding_batch_reg import collate_batch
import pandas as pd
import io
import numpy as np
import os
from add_ons.drop_column import drop_columns
from add_ons.dif_seq_candles import add_label_normalized_candles
from add_ons.feature_pipeline3 import FeaturePipeline
from sklearn.metrics import accuracy_score, f1_score
# ---------------- Evaluation ---------------- #
@torch.no_grad()
def evaluate_model_mdn(model, val_loader, zero_idx=0, threshold=0.1):
    """
    Evaluate CNN–LSTM–MDN model (last-output version).

    Args
    ----
    model : pl.LightningModule with MDN forward
    val_loader : DataLoader yielding (X, y, lengths)
    zero_idx : which mixture component is considered "no-line" (usually 0)
    threshold : if pi[:,zero_idx] > threshold → predict invalid

    Returns
    -------
    dict with mse, mae, acc, f1
    """
    model.eval()
    all_preds_reg, all_labels_reg = [], []
    all_preds_len, all_labels_len = [], []

    device = next(model.parameters()).device

    with torch.no_grad():
        for X_batch, y_batch, lengths in val_loader:
            if isinstance(X_batch, dict):
                X_batch = {k: v.to(device) for k, v in X_batch.items()}
            else:
                X_batch = X_batch.to(device)

            y_batch = y_batch.to(device)
            mdn = model(X_batch, lengths)
            pi, mu, sigma = mdn["pi"], mdn["mu"], mdn["sigma"]  # (B,K)

            # regression expectation
            y_pred = (pi * mu).sum(dim=-1)  # (B,)
            B = y_batch.size(0)
            y_len = (y_batch > 0).sum(dim=1)                # (B,)
            idx = torch.clamp(y_len - 1, min=0)             # last valid index
            y_true = y_batch[torch.arange(B, device=y_batch.device), idx]  # (B,)
            # only last step
            # print("lengths(features):", lengths[:10])
            # print("lengths(labels):", y_len[:10])

            all_preds_reg.append(y_pred.cpu().numpy())
            all_labels_reg.append(y_true.cpu().numpy())

            # validity classification
            pi_zero = pi[:, zero_idx]  # (B,)
            pred_valid = (pi_zero < (1 - threshold)).long()
            true_valid = torch.ones_like(pred_valid)  # last step always valid

            all_preds_len.extend(pred_valid.cpu().numpy().tolist())
            all_labels_len.extend(true_valid.cpu().numpy().tolist())


        # ----- Regression metrics -----
    all_preds_reg = np.concatenate(all_preds_reg)  # (N,)
    all_labels_reg = np.concatenate(all_labels_reg)
    mse = ((all_preds_reg - all_labels_reg) ** 2).mean()
    mae = np.abs(all_preds_reg - all_labels_reg).mean()
    # ----- Validity metrics -----
    acc = accuracy_score(all_labels_len, all_preds_len)
    f1 = f1_score(all_labels_len, all_preds_len, average="macro")

    print("\n📊 Validation Metrics (MDN, last-output):")
    print(f"  Regression → MSE: {mse:.6f}, MAE: {mae:.6f}")
    print(f"  Validity   → Acc: {acc:.4f}, F1: {f1:.4f}")

    return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}
# ---------------- Train ---------------- #
def train_model(
    data_csv,
    labels_csv,
    model_out_dir="models/saved_models",
    do_validation=True,
    hidden_dim=128,
    num_layers=1,
    lr=0.001,
    batch_size=32,
    max_epochs=300,
    save_model=False,
    return_val_accuracy = True,
    test_mode = False
):

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_multireg_{timestamp}.pt"
    meta_out  = f"{model_out_dir}/lstm_meta_multireg_{timestamp}.pkl"

    pipeline = FeaturePipeline(
            steps=[
                lambda df: add_label_normalized_candles(df),
                   lambda df: drop_columns(df, ["open","high","low","close","volume"])
                   ],
            norm_methods={
                "main": {"upper_shadow": "robust","body":"standard", "lower_shadow": "standard",
                         "upper_body_ratio":"standard","upper_body_ratio":"standard",
                         "lower_body_ratio":"standard","upper_lower_body_ratio":"standard", "Candle_Color": "standard"
                         },
            },
            per_window_flags=[True, True] 
        )
    # Preprocess: pad linePrices and sequences
    if do_validation:
        train_ds, val_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=True,
            for_xgboost=False,
            debug_sample=True,
            feature_pipeline=pipeline
        )
    else:
        train_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=False,
            for_xgboost=False,
            debug_sample=False,
            feature_pipeline=pipeline
        )
        val_ds = None

    sample = train_ds[0][0]  # first sample's features
    if isinstance(sample, dict):  # multiple feature groups
        input_dim = sample['main'].shape[1]
    else:  # single tensor
        input_dim = sample.shape[1]

    model = CNNLSTM_MDN(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        lr=lr
    )

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
    val_loader = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate_batch) if val_ds else None
    
    # --- Debug / Test mode --- #
    if test_mode:
        from itertools import islice

        # Try to grab 3rd batch; if not available, take first
        try:
            batch = next(islice(iter(train_loader), 2, 3))
        except StopIteration:
            batch = next(iter(train_loader))

        X_batch_dict, y_batch, lengths = batch

        print("🔍 Debug batch:")
        if isinstance(X_batch_dict, dict):
            print("  Keys in X_batch:", list(X_batch_dict.keys()))
        print("  y_batch shape:", y_batch.shape)
        print("  First label in batch:", y_batch[0])

        # --- Track real column names for each feature group ---
        feature_names_dict = {}
        for name, X_batch in X_batch_dict.items():
            if name == "main":
                # Use actual feature columns after preprocessing
                feature_names_dict[name] = feature_cols
            else:
                # For extra feature groups, fallback to generic names
                feature_names_dict[name] = [f"{name}_{i}" for i in range(X_batch.shape[2])]

        dfs = []
        for name, X_batch in X_batch_dict.items():
            print(f"\nFeature group: {name}")
            print("  X_batch shape:", X_batch.shape)
            print("  First sequence in batch (first 5 steps):\n", X_batch[0][:])

            batch_size_, seq_len, feature_dim = X_batch.shape
            df_part = pd.DataFrame(
                X_batch.reshape(batch_size_ * seq_len, feature_dim).numpy(),
                columns=feature_names_dict[name]
            )
            dfs.append(df_part)

        # Combine all feature groups horizontally
        global df_seq
        df_seq = pd.concat(dfs, axis=1)
        print("\n✅ Combined df_seq shape:", df_seq.shape)
        print("✅ Column names in df_seq:", df_seq.columns.tolist())


    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        fast_dev_run=test_mode,
        gradient_clip_val=1.0,
        gradient_clip_algorithm="norm"
    )
    trainer.fit(model, train_loader, val_loader)

    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
            "input_dim": input_dim,
            "hidden_dim": hidden_dim,
            "num_layers": num_layers,
            "max_len_y": max_len_y,
            "feature_cols": feature_cols
        }, meta_out)
        print(f"✅ Model saved to {model_out}")
        print(f"✅ Meta saved to {meta_out}")
    # --- Evaluation --- #
    if do_validation:
        mse, mae, acc, f1 = evaluate_model_mdn(model, val_loader)
        if return_val_accuracy:
            return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}
        
if __name__ == "__main__":
    train_model(
        "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles_prop.csv",
        "/home/iatell/projects/meta-learning/data/seq_line_labels.csv",
        save_model=False,
        do_validation=True,
        test_mode = True
    )


--- Debugging pipeline.fit() ---
Columns available for fitting: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'upper_shadow', 'body', 'lower_shadow', 'Candle_Color', 'upper_body_ratio', 'lower_body_ratio', 'upper_lower_body_ratio']
✅ 'upper_shadow' column FOUND before fitting.
---------------------------------
  Attempting to normalize 'upper_shadow' with method 'robust'. Found in df: True
  Attempting to normalize 'body' with method 'standard'. Found in df: True
  Attempting to normalize 'lower_shadow' with method 'standard'. Found in df: True
  Attempting to normalize 'upper_body_ratio' with method 'standard'. Found in df: True
  Attempting to normalize 'lower_body_ratio' with method 'standard'. Found in df: True
  Attempting to normalize 'upper_lower_body_ratio' with method 'standard'. Found in df: True
  Attempting to normalize 'Candle_Color' with method 'standard'. Found in df: True
  Attempting to normalize 'upper_shadow' with method 'robust'. Found in df: True
  Attemp

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



=== DEBUG SAMPLE ===
Label (linePrices padded): [0.774234 0.941543 1.004005 0.       0.       0.      ]
Feature group: main, Shape: (56, 11)
Columns: ['upper_shadow', 'body', 'lower_shadow', 'Candle_Color', 'upper_body_ratio', 'lower_body_ratio', 'upper_lower_body_ratio', 'open_prop', 'high_prop', 'low_prop', 'close_prop']
[[-0.43279868 -0.3618772   0.46807465 -1.0472498  -0.5697781   0.31477502
  -0.79383653  0.80305094  0.8095325   0.74693364  0.78384095]
 [ 1.896245    1.1791093   0.12663852  0.9548821  -0.3857863  -0.6607218
   0.2079668   0.7839675   0.90648395  0.75513643  0.8597125 ]
 [ 0.5089243   0.17218113  0.24277814  0.9548821   0.31929123  0.5880437
  -0.41781333  0.86058474  0.89676327  0.8289499   0.8740301 ]
 [-0.06139557  0.08720617  1.3182539   0.9548821   0.24282499  2.2641795
  -0.7630081   0.8740301   0.89514875  0.81536096  0.8822335 ]
 [-0.08807323  1.738903    0.04165139  0.9548821  -0.7168349  -0.7486489
  -0.59019285  0.88223463  1.0062362   0.8553123   0.993


  | Name     | Type        | Params | Mode 
-------------------------------------------------
0 | conv1    | Conv1d      | 768    | train
1 | conv3    | Conv1d      | 2.2 K  | train
2 | bn1      | BatchNorm1d | 128    | train
3 | bn3      | BatchNorm1d | 128    | train
4 | lstm     | LSTM        | 132 K  | train
5 | mdn_head | Linear      | 1.9 K  | train
-------------------------------------------------
137 K     Trainable params
0         Non-trainable params
137 K     Total params
0.549     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trai

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.



📊 Validation Metrics (MDN, last-output):
  Regression → MSE: 1.205708, MAE: 1.087041
  Validity   → Acc: 1.0000, F1: 1.0000


In [3]:
df_seq = df_seq.loc[~(df_seq==0).all(axis=1)]
df_seq

,upper_shadow,body,lower_shadow,Candle_Color,upper_body_ratio,lower_body_ratio,upper_lower_body_ratio,open_prop,high_prop,low_prop,close_prop
0,1.321786,0.008634,-0.895926,0.954882,2.493226,-0.355042,2.547058,1.162368,1.228990,1.157517,1.167987
1,-0.414832,-0.821408,-0.418283,-1.047250,-0.696935,-0.703118,-0.597881,1.167369,1.178731,1.063608,1.088384
2,1.001162,-0.689979,-0.681345,-1.047250,-0.274219,-0.768598,1.677601,1.088532,1.140088,1.009280,1.023049
3,0.310094,0.279493,-0.393333,0.954882,-0.158510,-0.409299,-0.057559,1.023519,1.088113,0.997934,1.056417
4,0.543558,-0.498462,-0.145810,-1.047250,-0.240747,-0.404925,-0.175599,1.056417,1.094575,0.975182,1.010898
...,...,...,...,...,...,...,...,...,...,...,...
1784,-0.138058,0.037541,0.281985,0.954882,0.687875,2.264179,-0.668850,1.203305,1.217259,1.176171,1.207596
1785,-0.397556,-0.435193,-0.296570,-1.047250,-0.594945,-0.424679,-0.640537,1.207596,1.213499,1.172994,1.187862
1786,2.885888,-0.594436,-0.837304,-1.047250,0.446491,-0.819213,2.547058,1.187862,1.240469,1.156306,1.159945
1787,0.005538,-2.359263,0.327503,-1.047250,-0.726435,-0.751515,-0.626643,1.159945,1.171894,1.010522,1.039174


# server

In [28]:
# server_cnn_mdn.py
import glob
import joblib
import torch
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
# from models.LSTM.cnn_lstm_mdn import CNNLSTM_MDN  # <-- your updated "last-output" model

app = Flask(__name__)

# ---------------- Load model and meta ----------------
meta_path = glob.glob("models/saved_models/lstm_meta_multireg_*.pkl")[0]
state_path = glob.glob("models/saved_models/lstm_model_multireg*.pt")[0]

meta = joblib.load(meta_path)
FEATURES = meta['feature_cols']

def one_sample(df_slice):
    return df_slice[FEATURES].values.astype(np.float32)

def load_raw_data_serve(candle_csv, label_csv):
    candles = pd.read_csv(candle_csv, parse_dates=['timestamp'])
    labels  = pd.read_csv(label_csv, header=None,
                          names=['timestamp', 'last_close', 'line_raw'])
    labels['timestamp'] = pd.to_datetime(labels['timestamp'])

    df = pd.merge(candles, labels[['timestamp', 'line_raw']], on='timestamp', how='left')
    df['label_price'] = df['line_raw'].str.extract(r"\(([^)]+)\)")[0].astype(float)
    df['has_line'] = (~df['label_price'].isna()).astype(int)
    df['log_factor'] = np.where(df['has_line'],
                                np.log(np.maximum(df['label_price'], 1e-6)),
                                np.nan)
    return df

# ---------------- Model ----------------
model = CNNLSTM_MDN(
    input_dim=meta['input_dim'],
    hidden_dim=meta.get('hidden_dim', 128),
    num_layers=meta.get('num_layers', 1),
    n_components=meta.get('n_components', 5),
    cnn_channels=meta.get('cnn_channels', 64),
    dropout=meta.get('dropout', 0.1)
)
model = CNNLSTM_MDN.load_from_checkpoint(state_path)
model.eval()

# ---------------- Load data ----------------
df = load_raw_data_serve(
    "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles_prop.csv",
    "/home/iatell/projects/meta-learning/data/ohlcv_log(2).csv"
)

# ---------------- Routes ----------------
@app.route("/")
def home():
    return render_template("sequential.html")

@app.route("/candles")
def candles():
    dense = df.set_index('timestamp').asfreq('D').ffill()
    return jsonify([
        {'time': int(ts.timestamp()),
         'open': float(row.open),
         'high': float(row.high),
         'low': float(row.low),
         'close': float(row.close)}
        for ts, row in dense.iterrows()
    ])

@app.route("/predict", methods=['POST'])
def predict():
    data = request.get_json(force=True)
    idxs = data.get('idxs')        # list of indices
    seq_len = data.get('seq_len')  # int

    if not idxs or not isinstance(idxs, list):
        return jsonify({"error": "Provide 'idxs' as a list"}), 400
    if not seq_len or not isinstance(seq_len, int):
        return jsonify({"error": "Provide 'seq_len' as an int"}), 400

    all_pred_prices = []
    all_pred_sigmas = []
    all_pi = []

    for idx in idxs:
        seq_df = df.iloc[idx - seq_len + 1: idx + 1]
        last_close = seq_df.iloc[-1]['close']

        X_np = one_sample(seq_df)
        X_t = torch.from_numpy(X_np.astype(np.float32)).unsqueeze(0)
        with torch.no_grad():
            mdn_out = model(X_t)

        # Now outputs are (B,K) — no time dimension
        pi    = mdn_out['pi'][0].cpu().numpy()
        mu    = mdn_out['mu'][0].cpu().numpy()
        sigma = mdn_out['sigma'][0].cpu().numpy()

        all_pred_prices.append((last_close * (mu)).tolist())
        all_pred_sigmas.append((last_close * sigma).tolist())
        all_pi.append(pi.tolist())

    return jsonify({
        'pred_prices': all_pred_prices,
        'pred_sigmas': all_pred_sigmas,
        'pi': all_pi
    })

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Sep/2025 22:19:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:43] "GET /candles HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:10] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:10] "POST /predict HTTP/1.1" 200 -
